# SCADA Flow Rate Calculation Function

In [1]:
import pandas as pd
import os
import csv
import math

import re
import datetime
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Step 1: Load Scada & Legend Data

This section will cover loading the Scada data and the Legend data, from your local drive into the Python notebook.

Instructions
1. The lines below will load both the Lift Station CSV file and the Legend CSV file into a data frame. Simply click shift enter to run each line.

2. Stop when you reach Step 2.


## Read Scada data into a data frame

In [15]:
df = pd.read_csv('LiftStation.csv')

## Read Legend data into a data frame

In [3]:
legend = pd.read_csv('Lift Station Legend Cross Referance.csv')

# Step 2: Clean and Format the Data

This section is designed to clean the data for further analysis. In the following lines we will

    1. Remove redundant column from the legend data frame.
    
    2. Convert pump start and stop columns to datetime in the Lift Station data frame.
    
    3. Drop any rows containing NA's in the lift station data frame.
    
    4. Convert the pump tag index column to int in the lift station data frame.
    
    5. Delete any duplicate rows containing the same pump tag index and start / stop timestamp.

Instructions
1. Simply click shift enter on the lines below this cell until you reach step 3.

## Clean the Legend data

In [4]:
legend = legend.drop('Pump #1 Run', axis=1)
legend = legend.drop('Pump #2 Run', axis=1)
legend = legend.drop('Pump #3 Run', axis=1)
legend = legend.drop('Table Tag #.4', axis=1)
legend = legend.drop('Pump #4 Run', axis=1)
legend = legend.drop('Unnamed: 11', axis=1)

## Convert Scada DateAndTime column to Datetime Format

In [5]:
df['PumpStart'] = pd.to_datetime(df['PumpStart'])
df['PumpStop'] = pd.to_datetime(df['PumpStop'])

## Format the dataframe

Drop any rows without a timestamp in the pump start and pump stop columns.

In [6]:
df = df.dropna(subset=['PumpStart', 'PumpStop'])

Convert the pump number column to integer format.

In [7]:
df['PumpTagIndex'] = df['PumpTagIndex'].astype(int)

Check for and remove duplicate entries with matching pump numbers and pump start and stop times.

In [8]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStart" values
mask = df.duplicated(subset =['PumpTagIndex', 'PumpStart'], keep = 'first')

# drop the duplicate rows using the boolean mask
new_df = df[~mask]

# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

In [9]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStop" values
mask = new_df.duplicated(subset=['PumpTagIndex', 'PumpStop'], keep='first')

# drop the duplicate rows using the boolean mask
new_df = new_df[~mask]

# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

# Step 3: Initializing two Data Frames and the Flow Rate Function

Step 3 will create two data frames to store the results of the function in. Calc_df will hold all of our valid calculation rows. Same time pumps df will hold all rows that are redundant to the model. It will also cover initializing the function for use in step 4.

Instructions
1. Click the play buttons on all the sections below until you reach step 3.

## Create a data frame to store function results: Calc_df

In [10]:
calc_df = pd.DataFrame()
calc_df['LiftStation'] = ()
calc_df['PumpTagIndex'] = ()
calc_df['PumpStartTime'] = ()
calc_df['StartWellLevel'] = ()
calc_df['PumpStopTime'] = ()
calc_df['StopWellLevel'] = ()
calc_df['StartUnix'] = ()
calc_df['StopUnix'] = ()
calc_df['Time_diff'] = ()
calc_df['Elevation_dif'] = ()
calc_df['Wet_well_diam'] = ()
calc_df['volume_per_foot'] = ()
calc_df['Observed_pumping_flow_rate'] = ()
calc_df['Avg_flow_rate_pump'] = int()
calc_df['Average_observed_pumping_flow_rate'] = ()
calc_df['Time_to_Fill'] = ()
calc_df['Inflow'] = ()
calc_df['Avg_Inflow'] = ()
calc_df['Actual_Pump_Flow_Rate'] = ()
calc_df['Average_Pump_Flow_Rate'] = ()
calc_df['Average_Pump_Flow_Rate_(all)'] = ()

## Create a data frame to store redundant function results: Same Time Pumps

In [11]:
same_time_pumps = pd.DataFrame()
same_time_pumps['LiftStation'] = ()
same_time_pumps['PumpTagIndex'] = ()
same_time_pumps['PumpStartTime'] = ()
same_time_pumps['StartWellLevel'] = ()
same_time_pumps['PumpStopTime'] = ()
same_time_pumps['StopWellLevel'] = ()
same_time_pumps['StartUnix'] = ()
same_time_pumps['StopUnix'] = ()
same_time_pumps['Time_diff'] = ()
same_time_pumps['Elevation_dif'] = ()
same_time_pumps['Wet_well_diam'] = ()
same_time_pumps['volume_per_foot'] = ()
same_time_pumps['Observed_pumping_flow_rate'] = ()
same_time_pumps['Avg_flow_rate_pump'] = int()
same_time_pumps['Average_observed_pumping_flow_rate'] = ()

# Flow Rate Function

In [12]:
def flow_rate_calculation (new_df):
    
    #create the calculated dataframe to store our flow rate data
    calc_df = new_df
    calc_df = calc_df.rename(columns={'PumpStart': 'PumpStartTime'})
    calc_df = calc_df.rename(columns={'PumpStop': 'PumpStopTime'})
    calc_df = calc_df.rename(columns={'WellStart': 'StartWellLevel'})
    calc_df = calc_df.rename(columns={'WellEnd': 'StopWellLevel'})
    calc_df['LiftStation'] = None
    calc_df['StartUnix'] = 0.0
    calc_df['StopUnix'] = 0.0
    calc_df['Time_diff'] = 0.0
    
    #arrange columns
    calc_df = calc_df[['LiftStation','PumpTagIndex', 'PumpStartTime', 'StartWellLevel','PumpStopTime', 'StopWellLevel', 'StartUnix', 'StopUnix','Time_diff']]
    
    #Delete any rows with incorrect pump numbers
    
    #create a list of valid pump numbers

    valid_pumps = []

    for index, row in legend.iterrows():
        valid_pumps.append(row['Table Tag #'])
        valid_pumps.append(row['Table Tag #.1'])
        valid_pumps.append(row['Table Tag #.2'])
        valid_pumps.append(row['Table Tag #.3'])

    valid_pumps = list(filter(lambda x: not math.isnan(x), valid_pumps))
    
    calc_df = calc_df[calc_df['PumpTagIndex'].isin(valid_pumps)]
    calc_df = calc_df.reset_index(drop=True)
    calc_df['PumpTagIndex'] = calc_df['PumpTagIndex'].astype(int)
    
    #Add the correct Lift Station to each row in calc_df
    
    #First Left Join (Table Tag 0)
    
    # Left join calc_df to legend df where Pump Tag Index = Table Tag
    merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #', how='left')

    # Create a mask to filter the rows where Lift Station is not empty
    mask = merged_df['Lift Station'].notnull()

    # Assign the Lift Station values to the LiftStation column in the calc_df dataframe
    calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

    #Second Left Join (Table Tag 1)
    # Left join calc_df to legend df where Pump Tag Index = Table Tag #.1
    merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.1', how='left')

    # Create a mask to filter the rows where Lift Station is not empty
    mask = merged_df['Lift Station'].notnull()

    # Assign the Lift Station values to the LiftStation column in the calc_df dataframe
    calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

    #Third Left Join (Table Tag 2)
    # Left join calc_df to legend df where Pump Tag Index = Table Tag #.2
    merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.2', how='left')

    # Create a mask to filter the rows where Lift Station is not empty
    mask = merged_df['Lift Station'].notnull()

    # Assign the Lift Station values to the LiftStation column in the calc_df dataframe
    calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

    #Third Left Join (Table Tag 3)
    # Left join calc_df to legend df where Pump Tag Index = Table Tag #.3
    merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.3', how='left')

    # Create a mask to filter the rows where Lift Station is not empty
    mask = merged_df['Lift Station'].notnull()

    # Assign the Lift Station values to the LiftStation column in the calc_df dataframe
    calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']
    
    #Now I'll remove unneeded lift stations
    calc_df = calc_df.drop(calc_df[calc_df['LiftStation'].isin(['Stratford 2662 (20)', 'Linda Vista Lift Station (5)', 'Linda Vista Golf Course (12)', 'Reserve Street (14)'])].index)
    calc_df = calc_df.reset_index(drop=True)

    #Now let's get Unix time for each timestamp
    calc_df['StartUnix'] = pd.to_datetime(calc_df['PumpStartTime']).astype(int)/ 10**9
    calc_df['StopUnix'] = pd.to_datetime(calc_df['PumpStopTime']).astype(int) / 10**9
    
    #Next I'll calculate the time difference between Pumps turning on and off
    calc_df['Time_diff'] = (calc_df['StopUnix'] - calc_df['StartUnix']) / 60
    
    
    #Section 3 
    
    #Add in Elevation Difference
    elevation_dict = {}

    for index,row in legend.iterrows():
        tag = row['Lift Station']
        elevation_dict[tag] = row['Elevation Diff']

    for index, row in calc_df.iterrows():
        tag = row['LiftStation']
        calc_df.at[index,'Elevation_dif'] = elevation_dict[tag]
        
    #Calculate Volume Per Foot for each well
    #initialize new column
    calc_df['Wet_well_diam'] = 0.0

    # left join calc_df and legend df on lift station
    merged_df = calc_df.merge(legend, left_on = 'LiftStation', right_on = 'Lift Station', how = 'left')

    #create a boolean mask to find all rows where we have a wet well diameter
    mask = merged_df['Wet Well Diam'].notnull()

    #add the wet well diameter value from the legend dataframe to the calc_df where mask is true
    calc_df.loc[mask, 'Wet_well_diam'] = merged_df.loc[mask, 'Wet Well Diam']
    
    #Calculate volume per foot
    calc_df['volume_per_foot'] = round(((calc_df['Wet_well_diam']*calc_df['Wet_well_diam']) * 3.14)/4*7.48)
    
    #Calculate Observed Pumping Flow Rate
    
    calc_df['Observed_pumping_flow_rate'] = int()
    for index, row in calc_df.iterrows():
        result = round((row['Elevation_dif'] * row['volume_per_foot'])/row['Time_diff'])
        calc_df.at[index,'Observed_pumping_flow_rate'] = result
        
    #Calculate Average Flow Rate Per Pump
    calc_df['Avg_flow_rate_pump'] = int()
    
    #Create two dictionaries
    #1. Dictionary containing a running total of Observed Pumping Flow Rate for each pump
    #2. Dictionary containing a running row count of Observed Pumping Flow Rate for each pump
    
    sum_dict ={}
    count_dict ={}

    for item in valid_pumps:
        result = 0
        sum_dict[item] = result
        count_dict[item] = result
        
    #Calculate Average Flow Rate Per Pump
    for index, row in calc_df.iterrows():
        tag = row["PumpTagIndex"]
        sum_dict[tag] += row['Observed_pumping_flow_rate']
        count_dict[tag] += 1
        calc_df.at[index, 'Avg_flow_rate_pump'] = sum_dict[tag] / count_dict[tag]    
        
    #Finally Let's calculate our Average Observed Pumping Flow Rate (GPM)
    calc_df['Average_observed_pumping_flow_rate'] = int()
    
    #Create a dictionary for sum and count for each Lift Station
    sum_dict ={}
    count_dict ={}

    for index, row in legend.iterrows():
        result = 0
        tag = row['Lift Station']
        sum_dict[tag] = result
        count_dict[tag] = result
        
    #Calculate Average Observed Pumping Flow Rate (GPM)
    for index, row in calc_df.iterrows():
        tag = row["LiftStation"]
        sum_dict[tag] += row['Observed_pumping_flow_rate']
        count_dict[tag] += 1
        calc_df.at[index, 'Average_observed_pumping_flow_rate'] = sum_dict[tag] / count_dict[tag]
        
    
    #Section 4
    
    #Calculate time to fill
    calc_df['Time_to_Fill'] = float
   
    # Sort the dataframe by LiftStation and PumpStartTime
    calc_df = calc_df.sort_values(by=['LiftStation', 'PumpStartTime'])
    calc_df = calc_df.reset_index(drop=True)
    
    #Calculate time to fill and remove redundant rows
    same_time_pumps = pd.DataFrame(columns=calc_df.columns)
    rows_to_drop = []

    for index, row in calc_df.iterrows():
        if index == 0:
            calc_df.at[index, 'Time_to_Fill'] = 0
            continue

        if row['LiftStation'] == calc_df.iloc[index-1]['LiftStation']:
            result = (row['StartUnix'] - calc_df.iloc[index-1]['StopUnix'])/60
            if result > 1.0:
                calc_df.at[index, 'Time_to_Fill'] = result
            else:
                rows_to_drop.append(index-1)
                rows_to_drop.append(index)
                same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
                same_time_pumps = same_time_pumps.append(calc_df.iloc[index])

        if row['LiftStation'] != calc_df.iloc[index-1]['LiftStation']:
            result = 0
            calc_df.at[index, 'Time_to_Fill'] = result

    # Drop the rows to be removed from calc_df
    calc_df.drop(index=rows_to_drop, inplace=True)
    # Reset the index of calc_df after dropping rows
    calc_df = calc_df.reset_index(drop=True) 
    
    #Calculate Inflow
    calc_df['Inflow'] = float()
    calc_df['Avg_Inflow'] = float()
    
    for index, row in calc_df.iterrows():
        if row['Time_to_Fill'] > 0:
            inflow = row['volume_per_foot'] * row['Elevation_dif'] / row['Time_to_Fill']
            calc_df.at[index, 'Inflow'] = inflow
        else:
            pass
    
    #Calculate Average Inflow
    sum_dict ={}
    count_dict ={}

    for index, row in legend.iterrows():
        result = 0
        tag = row['Lift Station']
        sum_dict[tag] = result
        count_dict[tag] = result

    for index, row in calc_df.iterrows():
        if row['Time_to_Fill'] > 0:
            tag = row['LiftStation']
            sum_dict[tag] += row['Inflow']
            count_dict[tag] += 1
            calc_df.at[index, 'Avg_Inflow'] = sum_dict[tag] / count_dict[tag]
            
    #Section 5
    
    #Calculate Actual Pump Flow Rate (GPM)
    calc_df['Actual_Pump_Flow_Rate'] = float()
    
    for index, row in calc_df.iterrows():
        result = round(row['Observed_pumping_flow_rate'] + row['Inflow'])
        calc_df.at[index, 'Actual_Pump_Flow_Rate'] = result


    #Calculate Average Pump Flow Rate per pump(GPM)
    calc_df['Average_Pump_Flow_Rate'] = int()
    calc_df['Average_Pump_Flow_Rate_(all)'] = int()
    
    pump_sum_dict={}
    pump_count_dict={}

    for item in valid_pumps:
        pump_sum_dict[item]=0
        pump_count_dict[item]=0

    for index, row in calc_df.iterrows():
        tag = row['PumpTagIndex']
        pump_sum_dict[tag] += row['Actual_Pump_Flow_Rate']
        pump_count_dict[tag] += 1
        calc_df.at[index,'Average_Pump_Flow_Rate'] = pump_sum_dict[tag] / pump_count_dict[tag]
        
    #Calculate Average Pump Flow Rate (all pumps)
    sum_dict ={}
    count_dict ={}

    for index, row in legend.iterrows():
        result = 0
        tag = row['Lift Station']
        sum_dict[tag] = result
        count_dict[tag] = result

    for index, row in calc_df.iterrows():
        tag = row['LiftStation']
        sum_dict[tag] += row['Actual_Pump_Flow_Rate']
        count_dict[tag] += 1
        calc_df.at[index,'Average_Pump_Flow_Rate_(all)'] = sum_dict[tag] / count_dict[tag]
    
    # Important this needs to be changed for Plug In 2 to run. When the user wants to run Plug In 2,
    # put a # in front of return(calc_df). Remove the # from in front of return(same_time_pumps).
    # Once you have made this change click the play button above to re initialize the function.
    # You are now ready to run Plug In 2.

    return(calc_df)
    #return(same_time_pumps)
    
    

# Step 4: Running the Flow Rate Function

Instructions

You are now ready to run the function using the loaded SCADA data, and the lift station specifics from the table above.

Important: There are plug in options below. The first plug in returns the calculated data frame that contains all flow rate data. Plug in 2 returns the same time pumps dataframe, which contains all of the rows with negative values or values underneath 1 minute in the time to fill column.

1. Click the play icon on the line beneath the Plug In 1 text. The function will begin to run (this will take 10-15 minutes).

2. Once the Plug In 1 is finished running, to run Plug In 2 you will need to make one change to the function above.

3. In the function above on line 282, put a # in front of return(calc_df). Remove the # from in front of return(same_time_pumps). Once you have made this change click the play button on the function box again, to re initialize the function. You are now ready to run Plug In 2.

Note: Red colored messages will be displayed when the function runs, this is not an error.

## Plug In 1

In [17]:
calc_df = calc_df.append(flow_rate_calculation(new_df))

In [14]:
calc_df

,LiftStation,PumpTagIndex,PumpStartTime,StartWellLevel,PumpStopTime,StopWellLevel,StartUnix,StopUnix,Time_diff,Elevation_dif,...,volume_per_foot,Observed_pumping_flow_rate,Avg_flow_rate_pump,Average_observed_pumping_flow_rate,Time_to_Fill,Inflow,Avg_Inflow,Actual_Pump_Flow_Rate,Average_Pump_Flow_Rate,Average_Pump_Flow_Rate_(all)
0,Buena Vista (19),920.0,2022-06-24 00:11:33,3.200578,2022-06-24 00:13:32,2.189985,1.656029e+09,1.656030e+09,1.983333,1.0,...,211.0,106.0,106.000000,106.000000,0,0.000000,0.000000,106.0,106.000000,106.000000
1,Buena Vista (19),918.0,2022-06-24 03:33:31,3.196136,2022-06-24 03:35:31,2.187765,1.656042e+09,1.656042e+09,2.000000,1.0,...,211.0,106.0,106.000000,106.000000,199.983333,1.055088,1.055088,107.0,107.000000,106.500000
2,Buena Vista (19),920.0,2022-06-24 06:24:22,3.198357,2022-06-24 06:26:18,2.232186,1.656052e+09,1.656052e+09,1.933333,1.0,...,211.0,109.0,107.500000,107.000000,168.85,1.249630,1.152359,110.0,108.000000,107.666667
3,Buena Vista (19),918.0,2022-06-24 07:32:38,3.200578,2022-06-24 07:34:38,2.229965,1.656056e+09,1.656056e+09,2.000000,1.0,...,211.0,106.0,106.000000,106.750000,66.333333,3.180905,1.828541,109.0,108.000000,108.000000
4,Buena Vista (19),920.0,2022-06-24 08:49:01,3.196136,2022-06-24 08:50:59,2.192206,1.656061e+09,1.656061e+09,1.966667,1.0,...,211.0,107.0,107.333333,106.800000,74.383333,2.836657,2.080570,110.0,108.666667,108.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348288,Waldo (22),1033.0,2023-01-29 23:14:11,4.922175,2023-01-29 23:15:17,3.814686,1.675034e+09,1.675034e+09,1.100000,1.0,...,376.0,342.0,368.580268,380.361486,58.866667,6.387316,2.059379,348.0,345.712042,342.235450
348289,Waldo (22),1032.0,2023-01-30 08:24:31,4.919978,2023-01-30 08:25:38,3.812489,1.675067e+09,1.675067e+09,1.116667,1.0,...,376.0,337.0,392.289608,380.324895,549.233333,0.684591,2.058167,338.0,338.683274,342.231718
348290,Waldo (22),1033.0,2023-01-30 11:15:06,5.019593,2023-01-30 11:16:12,3.816151,1.675077e+09,1.675077e+09,1.100000,1.0,...,376.0,342.0,368.535893,380.292580,169.466667,2.218725,2.058308,344.0,345.709059,342.233275
348291,Waldo (22),1032.0,2023-01-30 17:08:39,4.926570,2023-01-30 17:09:47,3.818348,1.675099e+09,1.675099e+09,1.133333,1.0,...,376.0,332.0,392.187075,380.251896,352.45,1.066818,2.057435,333.0,338.673179,342.225154


## Plug In 2

In [16]:
same_time_pumps = same_time_pumps.append(flow_rate_calculation(new_df))

In [ ]:
#same_time_pumps

# Step 5: Exporting the Data

This final section will export both of the dataframes to csv files for further use.
1. Simply click the play button on both of the export lines below to export csv's to your local hard drive.

## Export the data to CSV

In [17]:
calc_df.to_csv("Flow Rate Data for all Lift Stations.csv")

In [ ]:
same_time_pumps.to_csv("Pumps Running at the same time.csv")